<a href="https://colab.research.google.com/github/tinySculpture/STINTSY-Project/blob/master/Project_Labor_Force_Survey-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

## Labor Force Survey 2016

Taken from the Philippine Statistics Authority (PSA), the dataset is a survey conducted in 2016 with respect in Demographic, Economic Characteristics and Past Quarter Activities of the surveyed individuals in this dataset. (To be enhanced after doing the project code and stuff)

## Task

Create a machine learning model for a classification task with the target variable: `PUFC23_PCLASS`

# Description of the Dataset

TBA

# Python libraries and modules used.

Imported the necessary libraries
- pandas, numpy, matplotlib.pyplot, and seaborn.


# Data Preprocessing

TBA

In [1]:
from itertools import combinations
%load_ext autoreload
%autoreload 2

# Import libraries
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport

# Import custom modules
import data_analysis

sns.set_theme("paper", "white")

c:\Users\cruzg\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read dataset

In [2]:
df = pd.read_csv("LFS PUF April 2016.CSV")

> ⚠ **Run these when using Google Colab**

In [1]:
# Run for google colab
!git clone https://github.com/tinySculpture/STINTSY-Project.git
%cd STINTSY-Project/

Cloning into 'STINTSY-Project'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 36 (delta 9), reused 16 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 9.77 MiB | 3.64 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Updating files: 100% (14/14), done.
/content/STINTSY-Project


In [2]:
!pip install ydata-profiling --quiet

  You can safely remove it manually.
  You can safely remove it manually.


In [8]:
report = ProfileReport(df)
report.to_file('report.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 41.07it/s]


In [ ]:
from IPython.core.display import display, HTML
display(HTML('report.html'))

> ⚠ End of Cells for Google Colab

### Data Cleaning

1. Force Missing Values to NaN to correctly identify missing values because of different data types. (Int, Float, and Object)
2. Delete duplicated rows (keep the first instance)

There are missing data that are represented as different invisible characters - replace them with an empty string

In [25]:
# df.replace(["", " ", "NA", "N/A", "-", "NULL"], np.nan, inplace=True)
df.replace("", np.nan, inplace=True)

In [26]:
df = df.map(lambda x: np.nan if isinstance(x, str) and x.strip() == "" else x)

In [27]:
print("Duplicates before dropping: ", df.duplicated().sum())
df = df.drop_duplicates()
print("Duplicates after dropping: ", df.duplicated().sum())

Duplicates before dropping:  0
Duplicates after dropping:  0


### Sanity Checking
Check if missing data is properly replaced with NaN

In [28]:
df

,PUFREG,PUFPRV,PUFPRRCD,PUFHHNUM,PUFURB2K10,PUFPWGTFIN,PUFSVYMO,PUFSVYYR,PUFPSU,PUFRPL,...,PUFC33_WEEKS,PUFC34_WYNOT,PUFC35_LTLOOKW,PUFC36_AVAIL,PUFC37_WILLING,PUFC38_PREVJOB,PUFC40_POCC,PUFC41_WQTR,PUFC43_QKB,PUFNEWEMPSTAT
0,1,28,2800,1,2,405.2219,4,2016,217,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,01,1
1,1,28,2800,1,2,388.8280,4,2016,217,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,01,1
2,1,28,2800,1,2,406.1194,4,2016,217,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,01,1
3,1,28,2800,2,2,405.2219,4,2016,217,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,01,1
4,1,28,2800,2,2,384.3556,4,2016,217,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,96,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180857,17,59,5900,40880,2,239.4341,4,2016,258,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,50,1
180858,17,59,5900,40880,2,189.8885,4,2016,258,1,...,NaN,8,NaN,NaN,NaN,2,NaN,NaN,NaN,3
180859,17,59,5900,40880,2,207.7395,4,2016,258,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180860,17,59,5900,40880,2,207.7395,4,2016,258,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Impute missing values

- Fill numerical columns with their *mean*.
- Fill categorical values with their *mode*.

In [30]:
# Replace numeric values with mean
df.fillna(df.select_dtypes(include=["number"]).mean(numeric_only=True), inplace=True)

# Replace categorical values with mode
for col in df.select_dtypes(include=["object"]):
    mode_value = df[col].mode()[0] if not df[col].mode().empty else "unknown"
    df[col] = df[col].fillna(mode_value)

df

,PUFREG,PUFPRV,PUFPRRCD,PUFHHNUM,PUFURB2K10,PUFPWGTFIN,PUFSVYMO,PUFSVYYR,PUFPSU,PUFRPL,...,PUFC33_WEEKS,PUFC34_WYNOT,PUFC35_LTLOOKW,PUFC36_AVAIL,PUFC37_WILLING,PUFC38_PREVJOB,PUFC40_POCC,PUFC41_WQTR,PUFC43_QKB,PUFNEWEMPSTAT
0,1,28,2800,1,2,405.2219,4,2016,217,1,...,002,7,1,1,1,1,52,1,01,1
1,1,28,2800,1,2,388.8280,4,2016,217,1,...,002,7,1,1,1,1,52,1,01,1
2,1,28,2800,1,2,406.1194,4,2016,217,1,...,002,7,1,1,1,1,52,1,01,1
3,1,28,2800,2,2,405.2219,4,2016,217,1,...,002,7,1,1,1,1,52,1,01,1
4,1,28,2800,2,2,384.3556,4,2016,217,1,...,002,7,1,1,1,1,52,1,96,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180857,17,59,5900,40880,2,239.4341,4,2016,258,1,...,002,7,1,1,1,1,52,1,50,1
180858,17,59,5900,40880,2,189.8885,4,2016,258,1,...,002,8,1,1,1,2,52,1,01,3
180859,17,59,5900,40880,2,207.7395,4,2016,258,1,...,002,7,1,1,1,1,52,1,01,1
180860,17,59,5900,40880,2,207.7395,4,2016,258,1,...,002,7,1,1,1,1,52,1,01,1


### Delete Unnecessary Columns

- Delete constant values (Month, Year)
    - PUFSVYYR
    - PUFSVYMO
- Delete IDs
    - PUFPRRCD (-0.035)
    - PUFHHNUM (0.021)
    - PUFREG (0.020)
- Delete Columns with low correlation to target variable (Within -0.05 to +0.05)
    - PUF10_CONWR (0.037)
    - PUFC14_PROCC (-0.490)
    - PUFC25_PBASIC (0.035)
    - PUFC27_NJOBS (0.035)
    - PUFC31_FLWRK (0.026)
    - PUFC32_JOBSM (0.031)
    - PUFC33_WEEKS (-0.014)
    - PUFC34_WYNOT (-0.019)
    - PUFC35_LTLOOKW (0.026)
    - PUFC37_WILLING (0.046)
    - PUFPRV (-0.035)
    - PUFRPL (0.004)

In [31]:
drop_cols = [
    "PUFSVYYR", "PUFSVYMO", "PUFPRRCD", "PUFHHNUM", "PUFREG", # IDs, Geolocation
    # Low correlation
    "PUFC10_CONWR", "PUFC14_PROCC", "PUFC25_PBASIC", "PUFC27_NJOBS", "PUFC31_FLWRK", "PUFC32_JOBSM", "PUFC33_WEEKS", "PUFC34_WYNOT", "PUFC35_LTLOOKW", "PUFC37_WILLING", "PUFPRV", "PUFRPL"
]
df.drop(columns=drop_cols, inplace=True)

In [32]:
df

,PUFURB2K10,PUFPWGTFIN,PUFPSU,PUFHHSIZE,PUFC01_LNO,PUFC03_REL,PUFC04_SEX,PUFC05_AGE,PUFC06_MSTAT,PUFC07_GRADE,...,PUFC26_OJOB,PUFC28_THOURS,PUFC29_WWM48H,PUFC30_LOOKW,PUFC36_AVAIL,PUFC38_PREVJOB,PUFC40_POCC,PUFC41_WQTR,PUFC43_QKB,PUFNEWEMPSTAT
0,2,405.2219,217,3,1,1,1,49,2,350,...,1,032,1,2,1,1,52,1,01,1
1,2,388.8280,217,3,2,2,2,61,2,350,...,2,008,1,2,1,1,52,1,01,1
2,2,406.1194,217,3,3,3,1,19,1,350,...,1,034,1,2,1,1,52,1,01,1
3,2,405.2219,217,4,1,1,1,48,2,320,...,1,033,1,2,1,1,52,1,01,1
4,2,384.3556,217,4,2,2,2,41,2,350,...,2,072,2,2,1,1,52,1,96,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180857,2,239.4341,258,5,1,1,1,29,2,350,...,2,040,1,2,1,1,52,1,50,1
180858,2,189.8885,258,5,2,2,2,29,2,830,...,2,048,1,2,1,2,52,1,01,3
180859,2,207.7395,258,5,3,3,2,4,1,350,...,2,048,1,2,1,1,52,1,01,1
180860,2,207.7395,258,5,4,3,2,2,1,350,...,2,048,1,2,1,1,52,1,01,1


### Recheck EDA

Check if the data has been changed significantly, and if the statistical tests are still the same after imputation and feature reduction.

In [13]:
report = ProfileReport(df)
report.to_file('report-cleaned.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 34/34 [00:03<00:00, 10.44it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

#### Binning and Encoding

- One-hot encode categorical values

In [33]:
df = pd.get_dummies(df, columns=['PUFURB2K10'], drop_first=True)
df = df.rename(columns={"PUFURB2K10_2": "Rural_Area"})
df

,PUFPWGTFIN,PUFPSU,PUFHHSIZE,PUFC01_LNO,PUFC03_REL,PUFC04_SEX,PUFC05_AGE,PUFC06_MSTAT,PUFC07_GRADE,PUFC08_CURSCH,...,PUFC28_THOURS,PUFC29_WWM48H,PUFC30_LOOKW,PUFC36_AVAIL,PUFC38_PREVJOB,PUFC40_POCC,PUFC41_WQTR,PUFC43_QKB,PUFNEWEMPSTAT,Rural_Area
0,405.2219,217,3,1,1,1,49,2,350,1,...,032,1,2,1,1,52,1,01,1,True
1,388.8280,217,3,2,2,2,61,2,350,1,...,008,1,2,1,1,52,1,01,1,True
2,406.1194,217,3,3,3,1,19,1,350,2,...,034,1,2,1,1,52,1,01,1,True
3,405.2219,217,4,1,1,1,48,2,320,1,...,033,1,2,1,1,52,1,01,1,True
4,384.3556,217,4,2,2,2,41,2,350,1,...,072,2,2,1,1,52,1,96,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180857,239.4341,258,5,1,1,1,29,2,350,1,...,040,1,2,1,1,52,1,50,1,True
180858,189.8885,258,5,2,2,2,29,2,830,1,...,048,1,2,1,2,52,1,01,3,True
180859,207.7395,258,5,3,3,2,4,1,350,1,...,048,1,2,1,1,52,1,01,1,True
180860,207.7395,258,5,4,3,2,2,1,350,1,...,048,1,2,1,1,52,1,01,1,True


#### PUFC07_GRADE
This classification data contains a lot of values in the value set. It can be noticed that there are different values for highest educational attainment and the program category. Since they are only one column, it might be better to encode the highest educational attainment as an ordinal encoded variable, and the program category as 15 different categories instead of 50+.

In [34]:
education_ordinal = {
    0: 0, 10: 1, 210: 2, 220: 3, 230: 4, 240: 5, 250: 6, 260: 7,
    280: 8, 310: 9, 320: 10, 330: 11, 340: 12, 350: 13, # 350 = highschool graduate
    410: 14,
    420: 15, 810: 16, 820: 17, 830: 18, 840: 19, 900: 20 # 900 = college graduate
}

program_categories = {
    # High school graduates
    501: "Basic", 508: "Basic", 509: "Basic", 514: "Education",
    521: "Arts", 522: "Arts", 531: "Social Science", 532: "Social Science",
    534: "Business", 538: "Law", 542: "Sciences", 544: "Sciences",
    546: "Sciences", 548: "IT", 552: "Engineering", 554: "Engineering",
    558: "Engineering", 562: "Agriculture", 564: "Agriculture", 572: "Health",
    576: "Health", 581: "Health", 584: "Transport", 585: "Environment",
    586: "Security", 589: "Other",

    # College graduates
    601: "Basic", 614: "Education", 621: "Arts", 622: "Arts",
    631: "Social Science", 632: "Social Science", 634: "Business", 638: "Law",
    642: "Sciences", 644: "Sciences", 646: "Sciences", 648: "IT",
    652: "Engineering", 654: "Engineering", 658: "Engineering", 662: "Agriculture",
    664: "Agriculture", 672: "Health", 676: "Health", 681: "Health",
    684: "Transport", 685: "Environment", 686: "Security", 689: "Other"
}

def get_education_with_program(x):
    if x not in program_categories:
        return x

    if x in range(501, 590):
        return 350
    elif x in range(601, 690):
        return 900

df["Education"] = df["PUFC07_GRADE"].apply(get_education_with_program)
df["Program"] = df["PUFC07_GRADE"].astype(int).map(program_categories)
df["Program"] = df["Program"].fillna("Unknown")


### Check for Multicollinearity

Use Variance Inflation Factor (VIF) to check for multicollinearity. If the VIF is greater than 10, then the variable is highly correlated with other variables and should be removed.

In [29]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

test_numerical = df.select_dtypes(include=["number"])

vif_data = pd.DataFrame()
vif_data["feature"] = test_numerical.columns
vif_data["VIF"] = [variance_inflation_factor(test_numerical.values, i) for i in range(len(test_numerical.columns))]

vif_data

,feature,VIF
0,PUFPWGTFIN,4.426415
1,PUFPSU,3.557011
2,PUFHHSIZE,8.828200
3,PUFC01_LNO,10.105176
4,PUFC03_REL,5.743669
5,PUFC04_SEX,6.895639
6,PUFC05_AGE,3.483710


In [35]:
df = df.drop(columns=["PUFC01_LNO"])

In [36]:
test_numerical = df.select_dtypes(include=["number"])

vif_data = pd.DataFrame()
vif_data["feature"] = test_numerical.columns
vif_data["VIF"] = [variance_inflation_factor(test_numerical.values, i) for i in range(len(test_numerical.columns))]

vif_data

NameError: name 'variance_inflation_factor' is not defined

In [21]:
df.columns

Index(['PUFREG', 'PUFPRV', 'PUFPRRCD', 'PUFHHNUM', 'PUFURB2K10', 'PUFPWGTFIN',
       'PUFSVYMO', 'PUFSVYYR', 'PUFPSU', 'PUFRPL', 'PUFHHSIZE', 'PUFC01_LNO',
       'PUFC03_REL', 'PUFC04_SEX', 'PUFC05_AGE', 'PUFC06_MSTAT',
       'PUFC07_GRADE', 'PUFC08_CURSCH', 'PUFC09_GRADTECH', 'PUFC10_CONWR',
       'PUFC11_WORK', 'PUFC12_JOB', 'PUFC14_PROCC', 'PUFC16_PKB',
       'PUFC17_NATEM', 'PUFC18_PNWHRS', 'PUFC19_PHOURS', 'PUFC20_PWMORE',
       'PUFC21_PLADDW', 'PUFC22_PFWRK', 'PUFC23_PCLASS', 'PUFC24_PBASIS',
       'PUFC25_PBASIC', 'PUFC26_OJOB', 'PUFC27_NJOBS', 'PUFC28_THOURS',
       'PUFC29_WWM48H', 'PUFC30_LOOKW', 'PUFC31_FLWRK', 'PUFC32_JOBSM',
       'PUFC33_WEEKS', 'PUFC34_WYNOT', 'PUFC35_LTLOOKW', 'PUFC36_AVAIL',
       'PUFC37_WILLING', 'PUFC38_PREVJOB', 'PUFC40_POCC', 'PUFC41_WQTR',
       'PUFC43_QKB', 'PUFNEWEMPSTAT'],
      dtype='object')

# Initial Logistic Regression Model

In [50]:
df

,PUFPWGTFIN,PUFPSU,PUFHHSIZE,PUFC03_REL,PUFC04_SEX,PUFC05_AGE,PUFC06_MSTAT,PUFC07_GRADE,PUFC08_CURSCH,PUFC09_GRADTECH,...,PUFC30_LOOKW,PUFC36_AVAIL,PUFC38_PREVJOB,PUFC40_POCC,PUFC41_WQTR,PUFC43_QKB,PUFNEWEMPSTAT,Rural_Area,Education,Program
0,405.2219,217,3,1,1,49,2,350,1,2,...,2,1,1,52,1,01,1,True,350,Unknown
1,388.8280,217,3,2,2,61,2,350,1,2,...,2,1,1,52,1,01,1,True,350,Unknown
2,406.1194,217,3,3,1,19,1,350,2,2,...,2,1,1,52,1,01,1,True,350,Unknown
3,405.2219,217,4,1,1,48,2,320,1,2,...,2,1,1,52,1,01,1,True,320,Unknown
4,384.3556,217,4,2,2,41,2,350,1,2,...,2,1,1,52,1,96,1,True,350,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180857,239.4341,258,5,1,1,29,2,350,1,2,...,2,1,1,52,1,50,1,True,350,Unknown
180858,189.8885,258,5,2,2,29,2,830,1,2,...,2,1,2,52,1,01,3,True,830,Unknown
180859,207.7395,258,5,3,2,4,1,350,1,2,...,2,1,1,52,1,01,1,True,350,Unknown
180860,207.7395,258,5,3,2,2,1,350,1,2,...,2,1,1,52,1,01,1,True,350,Unknown


Libraries for the model

In [51]:
from logistic_model import train_logistic_regression, load_logistic_model
from sklearn.metrics import classification_report

In [52]:
df_copy = df.copy() # copy of original dataframe just to avoid any changes to original dataframe

Preparing data from data frame

In [53]:
features = ['PUFPWGTFIN', 'PUFPSU', 'PUFHHSIZE', 'PUFC03_REL', 'PUFC04_SEX',
       'PUFC05_AGE', 'PUFC06_MSTAT', 'PUFC07_GRADE', 'PUFC08_CURSCH',
       'PUFC09_GRADTECH', 'PUFC11_WORK', 'PUFC12_JOB', 'PUFC16_PKB',
       'PUFC17_NATEM', 'PUFC18_PNWHRS', 'PUFC19_PHOURS', 'PUFC20_PWMORE',
       'PUFC21_PLADDW', 'PUFC22_PFWRK', 'PUFC23_PCLASS', 'PUFC24_PBASIS',
       'PUFC26_OJOB', 'PUFC28_THOURS', 'PUFC29_WWM48H', 'PUFC30_LOOKW',
       'PUFC36_AVAIL', 'PUFC38_PREVJOB', 'PUFC40_POCC', 'PUFC41_WQTR',
       'PUFC43_QKB', 'PUFNEWEMPSTAT', 'Education', 'Rural_Area'] # removed 'Program' because it was causing error in model training
target = 'PUFC23_PCLASS'


Training the model (run once or conditionally). Also, the model will be saved in logistic_model.pkl

In [54]:
train_logistic_regression(df_copy, features, target)

c:\Users\cruzg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Model saved to logistic_model.pkl


Load the model and scaler

In [55]:
saved = load_logistic_model()
model = saved['model']
scaler = saved['scaler']


Using the model for prediction

In [ ]:
X_test_scaled = scaler.transform(df[features])  # example batch only
y_pred = model.predict(X_test_scaled)

Results

In [58]:
print(classification_report(df_copy[target], y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3323
           1       1.00      1.00      1.00    141556
           2       1.00      1.00      1.00      6327
           3       1.00      1.00      1.00     20788
           4       0.98      0.99      0.99      2328
           5       0.90      0.60      0.72       216
           6       0.99      1.00      1.00      6324

    accuracy                           1.00    180862
   macro avg       0.98      0.94      0.96    180862
weighted avg       1.00      1.00      1.00    180862



The classification report shows that the model achieved an overall accuracy of 1.00, indicating perfect performance on the dataset. However, this unusually high accuracy may suggest the presence of class imbalance or overfitting. Upon further inspection, the performance per class reveals the following:

* Class 1 is the dominant class with 141,556 support. It achieves perfect precision, recall, and F1-score, which is expected due to the large number of samples.

* Classes 0, 2, 3, 4, and 6 also demonstrate near-perfect performance with precision, recall, and F1-scores close to or equal to 1.00, indicating that the model performs very well on these classes.

* Class 5, however, is problematic. Despite having a precision of 0.90, it suffers from a low recall of 0.60, meaning the model fails to identify 40% of actual Class 5 instances. Its F1-score drops to 0.72, which is significantly lower than the other classes. This class also has very low support with only 216 samples, contributing to its poor performance.



There are two main issues as seen from the results. Firstly, Class 1 heavily outweighs all other classes, making the model biased toward predicting this class. This imbalance can skew performance metrics and mask poor detection of minority classes. On the other hand, Class 5 being a minority class with very low training samples struggles to generalize well on this class, as shown by the low recall, resulting in many false negatives.